In [32]:
from openfermion.hamiltonians import MolecularData, load_molecular_hamiltonian
from openfermion.ops import FermionOperator
from openfermion.transforms import bravyi_kitaev
from openfermion.utils import hermitian_conjugated
from openfermion.ops import QubitOperator
from forestopenfermion import pyquilpauli_to_qubitop, qubitop_to_pyquilpauli
from pyquil.paulis import *
from pyquil.api import WavefunctionSimulator
from scipy.optimize import minimize
from pyquil import Program
from pyquil.gates import *

import numpy as np
import functools

In [59]:
# Set parameters to make a simple molecule.
diatomic_bond_length = 0.7
geometry = [('H', (0., 0., -diatomic_bond_length)), ('Be', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1  # 2S+1
charge = 0
description = str(diatomic_bond_length)

# Make molecule and print out a few interesting facts about it.
#molecule = MolecularData(geometry, basis, multiplicity, charge, description)

h2_interaction_hamil = load_molecular_hamiltonian(geometry,
        basis,
        multiplicity,
        description,
        n_active_electrons=None,
        n_active_orbitals=None)

In [60]:
sim = WavefunctionSimulator(random_seed=1337)

In [61]:
print(h2_interaction_hamil)  # in 'Interaction Operator' format

() 6.425723272421427
((0, 1), (0, 0)) -9.357227121126414
((0, 1), (4, 0)) 0.3699777887320237
((0, 1), (10, 0)) 0.10702417143457466
((1, 1), (1, 0)) -9.357227121126414
((1, 1), (5, 0)) 0.3699777887320237
((1, 1), (11, 0)) 0.10702417143457466
((2, 1), (2, 0)) -3.0700417075660105
((2, 1), (12, 0)) -0.3436005597605176
((3, 1), (3, 0)) -3.0700417075660105
((3, 1), (13, 0)) -0.3436005597605176
((4, 1), (0, 0)) 0.3699777887320237
((4, 1), (4, 0)) -3.0761889300535366
((4, 1), (10, 0)) -0.413853813262519
((5, 1), (1, 0)) 0.3699777887320237
((5, 1), (5, 0)) -3.0761889300535366
((5, 1), (11, 0)) -0.413853813262519
((6, 1), (6, 0)) -2.5538383102626185
((7, 1), (7, 0)) -2.5538383102626185
((8, 1), (8, 0)) -2.553838310262617
((9, 1), (9, 0)) -2.553838310262617
((10, 1), (0, 0)) 0.10702417143457421
((10, 1), (4, 0)) -0.4138538132625198
((10, 1), (10, 0)) -2.029274883658898
((11, 1), (1, 0)) 0.10702417143457421
((11, 1), (5, 0)) -0.4138538132625198
((11, 1), (11, 0)) -2.029274883658898
((12, 1), (2, 0

In [62]:
from openfermion.ops import FermionOperator, SymbolicOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.utils import hermitian_conjugated

In [63]:
h2_fermionop_hamil = FermionOperator()

for key in h2_interaction_hamil:
    value = h2_interaction_hamil[key]
    h2_fermionop_hamil += FermionOperator(term=key, coefficient=value)

print(h2_fermionop_hamil)


6.425723272421427 [] +
-9.357227121126414 [0^ 0] +
1.124962699194967 [0^ 0^ 0 0] +
-0.15179443860008585 [0^ 0^ 0 4] +
-0.049535121758304525 [0^ 0^ 0 10] +
0.007799252731169183 [0^ 0^ 2 2] +
-0.002730461380905046 [0^ 0^ 2 12] +
-0.15179443860008585 [0^ 0^ 4 0] +
0.032894541246124745 [0^ 0^ 4 4] +
0.011830819026589506 [0^ 0^ 4 10] +
0.007887994004499722 [0^ 0^ 6 6] +
0.007887994004499719 [0^ 0^ 8 8] +
-0.04953512175830465 [0^ 0^ 10 0] +
0.01183081902658954 [0^ 0^ 10 4] +
0.005274081982954007 [0^ 0^ 10 10] +
-0.0027304613809050406 [0^ 0^ 12 2] +
0.0029959965605921403 [0^ 0^ 12 12] +
1.124962699194967 [0^ 1^ 1 0] +
-0.15179443860008585 [0^ 1^ 1 4] +
-0.049535121758304525 [0^ 1^ 1 10] +
0.007799252731169183 [0^ 1^ 3 2] +
-0.002730461380905046 [0^ 1^ 3 12] +
-0.15179443860008585 [0^ 1^ 5 0] +
0.032894541246124745 [0^ 1^ 5 4] +
0.011830819026589506 [0^ 1^ 5 10] +
0.007887994004499722 [0^ 1^ 7 6] +
0.007887994004499719 [0^ 1^ 9 8] +
-0.04953512175830465 [0^ 1^ 11 0] +
0.01183081902658954 [0^ 1

In [69]:
x = FermionOperator('0^')
print(jordan_wigner(x))

0.5 [X0] +
-0.5j [Y0]


In [72]:
h2_qubitop_hamil = jordan_wigner(h2_fermionop_hamil)
print(h2_qubitop_hamil)

(-6.557006316757752+0j) [] +
(-0.0038996263655845893+0j) [X0 X1 Y2 Y3] +
(0.0013652306904525322+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Y13] +
(0.0013652306904525322+0j) [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 X12] +
(-0.016447270623062366+0j) [X0 X1 Y4 Y5] +
(-0.005915409513294759+0j) [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
(-0.005915409513294759+0j) [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
(-0.003943997002249861+0j) [X0 X1 Y6 Y7] +
(-0.00394399700224986+0j) [X0 X1 Y8 Y9] +
(-0.0026370409914770075+0j) [X0 X1 Y10 Y11] +
(-0.001497998280296029+0j) [X0 X1 Y12 Y13] +
(0.0038996263655845893+0j) [X0 Y1 Y2 X3] +
(-0.0013652306904525322+0j) [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 X13] +
(0.0013652306904525322+0j) [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 X12] +
(0.016447270623062366+0j) [X0 Y1 Y4 X5] +
(0.005915409513294759+0j) [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
(-0.005915409513294759+0j) [X0 Y1 Y5 Z6 Z7 Z8 Z9 X10] +
(0.003943997002249861+0j) [X0 Y1 Y6 X7] +
(0.00394399700224986+0j) [X0 Y1 Y8 X9] +
(0.00263

In [74]:
from openfermion.ops import QubitOperator
from forestopenfermion import pyquilpauli_to_qubitop, qubitop_to_pyquilpauli

h2_pauliop_hamil = qubitop_to_pyquilpauli(h2_qubitop_hamil)
print(h2_pauliop_hamil)
print(len(h2_pauliop_hamil))
numGates=0
for term in h2_pauliop_hamil:
    numGates += len(term.pauli_string())
print(numGates)

(-6.557006316757752+0j)*I + (2.4189382442359766+0j)*Z0 + (0.06556925338609318+0j)*Y0*Z1*Z2*Z3*Y4 + (0.06556925338609318+0j)*X0*Z1*Z2*Z3*X4 + (0.03207913757505531+0j)*Y0*Z1*Z2*Z3*Z4*Z5*Z6*Z7*Z8*Z9*Y10 + (0.03207913757505531+0j)*X0*Z1*Z2*Z3*Z4*Z5*Z6*Z7*Z8*Z9*X10 + (2.418938244235978+0j)*Z1 + (0.06556925338609326+0j)*Y1*Z2*Z3*Z4*Y5 + (0.06556925338609326+0j)*X1*Z2*Z3*Z4*X5 + (0.0320791375750553+0j)*Y1*Z2*Z3*Z4*Z5*Z6*Z7*Z8*Z9*Z10*Y11 + (0.0320791375750553+0j)*X1*Z2*Z3*Z4*Z5*Z6*Z7*Z8*Z9*Z10*X11 + (0.06472694289281533+0j)*Z2 + (0.00026028609753205034+0j)*Y2*Z3*Z4*Z5*Z6*Z7*Z8*Z9*Z10*Z11*Y12 + (0.00026028609753205034+0j)*X2*Z3*Z4*Z5*Z6*Z7*Z8*Z9*Z10*Z11*X12 + (0.06472694289281544+0j)*Z3 + (0.00026028609753207376+0j)*Y3*Z4*Z5*Z6*Z7*Z8*Z9*Z10*Z11*Z12*Y13 + (0.00026028609753207376+0j)*X3*Z4*Z5*Z6*Z7*Z8*Z9*Z10*Z11*Z12*X13 + (0.08476706450221776+0j)*Z4 + (0.001837046852981578+0j)*Y4*Z5*Z6*Z7*Z8*Z9*Y10 + (0.001837046852981578+0j)*X4*Z5*Z6*Z7*Z8*Z9*X10 + (0.08476706450221753+0j)*Z5 + (0.00183704685298

In [56]:
from pyquil.paulis import *
operator = sZ(0)*sZ(1) + sX(2)*sY(3)
print(operator)
print(len(operator[0].pauli_string()))

(1+0j)*Z0*Z1 + (1+0j)*X2*Y3
2


In [57]:
from pyquil.paulis import PauliSum
from pyquil.api import WavefunctionSimulator
from scipy.optimize import minimize
from pyquil import Program
from pyquil.gates import *

import numpy as np
import functools

In [58]:
sim = WavefunctionSimulator(random_seed=1337)
numQubit = 12
numLayer = 1

def solve_vqe(hamiltonian: PauliSum) -> float:
    # Construct a variational quantum eigensolver solution to find the lowest
    # eigenvalue of the given hamiltonian
    theta_init = np.random.rand(2*numQubit*numLayer) * 2 * np.pi

    def ansatz_energy(theta_vec):
        p = None
        p = Program()
        for j in range(numLayer):
            for i in range(numQubit):
                p += RX(theta_vec[2*numQubit * j + 2 * i], i)
                p += RZ(theta_vec[2*numQubit * j + 2 * i + 1], i)
            for i in range(numQubit - 1):
                p += CNOT(i, i + 1)

        energy = sim.expectation(p, hamiltonian).real
        return energy

    option = {}
    option['disp']=True
    theta_answer = minimize(ansatz_energy, theta_init, options=option).x
    return ansatz_energy(theta_answer)

In [13]:
solve_vqe(h2_pauliop_hamil)

Optimization terminated successfully.
         Current function value: -7.679193
         Iterations: 38
         Function evaluations: 1196
         Gradient evaluations: 46


-7.679192994650173